In [1]:
%load_ext sql
import urllib.parse
from sqlalchemy import create_engine
import pandas as pd
import os

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření connection stringu
connection_url = "mssql+pyodbc:///?odbc_connect=%s" % params

# Nastavení připojení
%sql $connection_url

In [ ]:
%%sql
select * from [dbo].[Valuo_data]

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse
from opencage.geocoder import OpenCageGeocode

# API klíč pro získání GPS souřadnic
api_key = '85af71fbd7334627a5b84894066a8a18'
geocoder = OpenCageGeocode(api_key)
processed_addresses = {}

def get_coordinates(address):
    if not address or address == "none":
        return (None, None)
    
    if address in processed_addresses:
        return processed_addresses[address]
    
    try:
        result = geocoder.geocode(address)
        if result and len(result):
            latitude = result[0]['geometry']['lat']
            longitude = result[0]['geometry']['lng']
            processed_addresses[address] = (latitude, longitude)
            print(f"Získány souřadnice: {address} -> ({latitude}, {longitude})")
            return (latitude, longitude)
        else:
            print(f"Nenalezeny souřadnice pro adresu: {address}")
            return (None, None)
    except Exception as e:
        print(f"Chyba při získávání GPS souřadnic pro adresu '{address}': {e}")
        return (None, None)

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření connection stringu
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"

# Vytvoření SQLAlchemy engine
engine = create_engine(connection_url)

# Cesta ke složce s Excel soubory
directory = r"C:\\Users\\ijttr\\OneDrive\\Dokumenty\\PROG\\PYTHON\\DATA_ANALYSIS\\VALUO\\data"

# Projít všechny soubory ve složce
for filename in os.listdir(directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        file_path = os.path.join(directory, filename)
        print(f"Importuji soubor: {filename}")
        
        # Načtení dat z Excelu
        df = pd.read_excel(file_path, sheet_name=0)
        
        # Přemapování názvů sloupců podle SQL tabulky
        df.columns = [
            "cislo_vkladu", "datum_podani", "datum_zplatneni", "listina", "nemovitost", "typ", "adresa", 
            "cenovy_udaj", "mena", "plocha", "typ_plochy", "popis", "okres", "kat_uzemi", "rok", "mesic"
        ]
        
        # Odstranění duplicitních hodnot v rámci jednoho souboru
        df.drop_duplicates(subset=["cislo_vkladu"], keep="first", inplace=True)
        
        # Konverze datových typů
        df["datum_podani"] = pd.to_datetime(df["datum_podani"], errors='coerce')
        df["datum_zplatneni"] = pd.to_datetime(df["datum_zplatneni"], errors='coerce')
        df["cenovy_udaj"] = pd.to_numeric(df["cenovy_udaj"], errors='coerce').fillna(0)
        df["plocha"] = pd.to_numeric(df["plocha"], errors='coerce').fillna(0)
        df["rok"] = df["rok"].astype('Int64').fillna(0)
        df["mesic"] = df["mesic"].astype('Int64').fillna(0)
        
        # Nahrazení všech NULL hodnot v textových sloupcích "none"
        text_columns = ["cislo_vkladu", "listina", "nemovitost", "typ", "adresa", "mena", "typ_plochy", "popis", "okres", "kat_uzemi"]
        df[text_columns] = df[text_columns].fillna("none")
        
        # Přidání GPS souřadnic pouze pro budovy a jednotky s platnou adresou
        df["LAT"], df["LON"] = None, None
        mask = (df["nemovitost"].isin(["budova", "jednotka"])) & (df["adresa"] != "none")
        
        # Oprava: Rozdělení tuple do dvou sloupců správně
        coords = df.loc[mask, "adresa"].apply(lambda x: get_coordinates(x))
        df.loc[mask, "LAT"] = coords.apply(lambda x: x[0] if x is not None else None)
        df.loc[mask, "LON"] = coords.apply(lambda x: x[1] if x is not None else None)
        
        # Konverze GPS souřadnic na float, aby odpovídaly DECIMAL(9,7)
        df["LAT"] = df["LAT"].astype(float).fillna(0.0)
        df["LON"] = df["LON"].astype(float).fillna(0.0)
        
        # Kontrola, zda se souřadnice správně načetly
        print("Ukázka prvních přiřazených souřadnic:")
        print(df.loc[mask, ["adresa", "LAT", "LON"]].head())
        
        # Kontrola a odstranění duplicitních hodnot před vložením do DB
        existing_keys = pd.read_sql("SELECT cislo_vkladu FROM Valuo_data", engine)
        df = df[~df["cislo_vkladu"].isin(existing_keys["cislo_vkladu"])]
        
        if not df.empty:
            # Vložení dat do SQL tabulky
            df.to_sql("Valuo_data", engine, if_exists='append', index=False)
            print(f"Soubor {filename} byl úspěšně naimportován.")
        else:
            print(f"Soubor {filename} neobsahuje nové záznamy k importu.")

print("Všechny soubory byly zpracovány.")


In [53]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse
from opencage.geocoder import OpenCageGeocode

# API klíč pro získání GPS souřadnic
api_key = '85af71fbd7334627a5b84894066a8a18'
geocoder = OpenCageGeocode(api_key)
processed_addresses = {}

def get_coordinates(address):
    if not address or address == "none":
        return {"LAT": None, "LON": None}
    
    if address in processed_addresses:
        return processed_addresses[address]
    
    try:
        result = geocoder.geocode(address)
        if result and len(result):
            latitude = result[0]['geometry']['lat']
            longitude = result[0]['geometry']['lng']
            processed_addresses[address] = {"LAT": latitude, "LON": longitude}
            print(f"Získány souřadnice: {address} -> ({latitude}, {longitude})")
            return processed_addresses[address]
        else:
            print(f"Nenalezeny souřadnice pro adresu: {address}")
            return {"LAT": None, "LON": None}
    except Exception as e:
        print(f"Chyba při získávání GPS souřadnic pro adresu '{address}': {e}")
        return {"LAT": None, "LON": None}

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření connection stringu
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"

# Vytvoření SQLAlchemy engine
engine = create_engine(connection_url)

# Cesta ke složce s Excel soubory
directory = r"C:\\Users\\ijttr\\OneDrive\\Dokumenty\\PROG\\PYTHON\\DATA_ANALYSIS\\VALUO\\data"

# Načtení existujících adres s GPS souřadnicemi z databáze
existing_gps_records = pd.read_sql("SELECT adresa, LAT, LON FROM Valuo_data WHERE LAT IS NOT NULL AND LON IS NOT NULL AND LAT != 0 AND LON != 0", engine)
existing_gps_records = existing_gps_records.drop_duplicates(subset=["adresa"], keep="first")  # Odstranění duplicit adres
existing_gps_dict = existing_gps_records.set_index("adresa").to_dict("index")
processed_addresses.update(existing_gps_dict)

# Projít všechny soubory ve složce
for filename in os.listdir(directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        file_path = os.path.join(directory, filename)
        print(f"Importuji soubor: {filename}")
        
        # Načtení dat z Excelu
        df = pd.read_excel(file_path, sheet_name=0)
        
        # Přemapování názvů sloupců podle SQL tabulky
        df.columns = [
            "cislo_vkladu", "datum_podani", "datum_zplatneni", "listina", "nemovitost", "typ", "adresa", 
            "cenovy_udaj", "mena", "plocha", "typ_plochy", "popis", "okres", "kat_uzemi", "rok", "mesic"
        ]
        
        # Odstranění duplicitních hodnot v rámci jednoho souboru (pouze přesně shodné řádky)
        df.drop_duplicates(keep="first", inplace=True)
        
        # Konverze datových typů
        df["datum_podani"] = pd.to_datetime(df["datum_podani"], errors='coerce')
        df["datum_zplatneni"] = pd.to_datetime(df["datum_zplatneni"], errors='coerce')
        df["cenovy_udaj"] = pd.to_numeric(df["cenovy_udaj"], errors='coerce').fillna(0)
        df["plocha"] = pd.to_numeric(df["plocha"], errors='coerce').fillna(0)
        df["rok"] = df["rok"].astype('Int64').fillna(0)
        df["mesic"] = df["mesic"].astype('Int64').fillna(0)
        
        # Nahrazení všech NULL hodnot v textových sloupcích "none"
        text_columns = ["cislo_vkladu", "listina", "nemovitost", "typ", "adresa", "mena", "typ_plochy", "popis", "okres", "kat_uzemi"]
        df[text_columns] = df[text_columns].fillna("none")
        
        # Kontrola a odstranění existujících záznamů před importem
        existing_records = pd.read_sql("SELECT cislo_vkladu, nemovitost, typ, adresa, plocha, typ_plochy FROM Valuo_data", engine)
        merged = df.merge(existing_records, on=["cislo_vkladu", "nemovitost", "typ", "adresa", "plocha", "typ_plochy"], how="left", indicator=True)
        df = merged[merged["_merge"] == "left_only"].drop(columns=["_merge"], errors='ignore')
        
        if df.empty:
            print(f"Soubor {filename} neobsahuje nové záznamy k importu.")
            print('---------------------------------------------------------------------------------------------')
            continue
        
        # Přidání GPS souřadnic pouze pro budovy a jednotky s platnou adresou
        df["LAT"], df["LON"] = None, None
        mask = (df["nemovitost"].isin(["budova", "jednotka"])) & (df["adresa"] != "none")
        
        # Počet celkových adres k získání GPS
        total_gps_requests = mask.sum()
        
        # Načítání unikátních adres pro minimalizaci API dotazů
        unique_addresses = df.loc[mask, "adresa"].unique()
        unique_gps_requests = sum(1 for addr in unique_addresses if addr not in processed_addresses)
        
        address_coords = {addr: get_coordinates(addr) for addr in unique_addresses if addr not in processed_addresses}
        processed_addresses.update(address_coords)
        
        # Přiřazení GPS souřadnic zpět do tabulky
        df.loc[mask, "LAT"] = df.loc[mask, "adresa"].map(lambda x: processed_addresses.get(x, {"LAT": None, "LON": None})["LAT"])
        df.loc[mask, "LON"] = df.loc[mask, "adresa"].map(lambda x: processed_addresses.get(x, {"LAT": None, "LON": None})["LON"])
        
        # Výpis statistik GPS dotazů
        print(f"GPS CELKEM = {total_gps_requests} / GPS unikátních/získaných = {unique_gps_requests}")
        
        # Vložení dat do SQL tabulky
        df.to_sql("Valuo_data", engine, if_exists='append', index=False)
        print(f"Soubor {filename} byl úspěšně naimportován.")
        print('---------------------------------------------------------------------------------------------')
print("Všechny soubory byly zpracovány.")

Importuji soubor: cadastre_report_3_2018.xlsx
Soubor cadastre_report_3_2018.xlsx neobsahuje nové záznamy k importu.
---------------------------------------------------------------------------------------------
Importuji soubor: cadastre_report_4_2022.xlsx
Soubor cadastre_report_4_2022.xlsx neobsahuje nové záznamy k importu.
---------------------------------------------------------------------------------------------
Importuji soubor: cadastre_report_7_2023.xlsx


C:\Users\ijttr\AppData\Local\Temp\ipykernel_53076\1287455522.py:89: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  merged = df.merge(existing_records, on=["cislo_vkladu", "nemovitost", "typ", "adresa", "plocha", "typ_plochy"], how="left", indicator=True)


Získány souřadnice: Tyršovo náměstí 221, 26724 Hostomice -> (49.8250101, 14.0450088)
Získány souřadnice: Tyršovo náměstí 73, 26724 Hostomice -> (49.8248188, 14.04875)
Získány souřadnice: č.p. 76, 26725 Osov -> (49.84437, 14.08391)
Získány souřadnice: Tyršovo náměstí 27, 26724 Hostomice -> (49.8261776, 14.0447013)
Získány souřadnice: Skřipel 81 -> (49.8419587, 14.0669707)
Získány souřadnice: Hostomice pod Brdy 1075 -> (49.812518, 14.0503521)
Získány souřadnice: Osov 51/1 -> (49.8443729, 14.0839129)
Získány souřadnice: Osov 146 -> (49.8443729, 14.0839129)
Získány souřadnice: Osov 147/5 -> (49.8443729, 14.0839129)
Získány souřadnice: Osov 147/6 -> (49.8443729, 14.0839129)
Získány souřadnice: Osov 151 -> (49.8443729, 14.0839129)
Získány souřadnice: Osov 51/8 -> (49.8443729, 14.0839129)
Získány souřadnice: Osov 205 -> (49.8443729, 14.0839129)
Získány souřadnice: U Hřiště 541, 26724 Hostomice -> (49.8284925, 14.04767)
Získány souřadnice: Osov 155 -> (49.8443729, 14.0839129)
Získány souřadnic